# CARGA DE DATOS (MATRICES DISTANCIA Y TIEMPO)


## Distancias y tiempos terrestres


In [1]:
# load archivos to dataframes
import pandas as pd
import numpy as np
import requests

# Load the depots and clients data
depots_df = pd.read_csv('Depots.csv')
clients_df = pd.read_csv('Clients.csv')

# extract coordinates
depots_coords = depots_df[['Longitude', 'Latitude']].values.tolist()
clients_coords = clients_df[['Longitude', 'Latitude']].values.tolist()

# Combine depots and clients
all_coords = depots_coords + clients_coords


# Build the coordinates string for OSRM
coords_str = ';'.join([f"{lon},{lat}" for lon, lat in all_coords])
# API URL
url = f"https://router.project-osrm.org/table/v1/driving/{coords_str}"

# Parameters
params = {
    'sources': ';'.join(map(str, range(len(all_coords)))),
    'destinations': ';'.join(map(str, range(len(all_coords)))),
    'annotations': 'duration,distance'
}

# Send the request
response = requests.get(url, params=params)

# Check for successful response
if response.status_code != 200:
    print(f"Error: {response.status_code}")
    print(response.text)
    exit()

data = response.json()

# Extract the distance matrix
distance_matrix = np.array(data['distances'])

# Extract the duration matrix
duration_matrix = np.array(data['durations'])

depot_labels = [f"D{i+1}" for i in range(len(depots_coords))]
client_labels = [f"C{i+1}" for i in range(len(clients_coords))]
all_labels = depot_labels + client_labels

# Convertir las matrices en DataFrames con etiquetas
distance_df = pd.DataFrame(distance_matrix, index=all_labels, columns=all_labels)
duration_df = pd.DataFrame(duration_matrix, index=all_labels, columns=all_labels)

# Guardar las matrices en archivos CSV
distance_df.to_csv('distance_matrix.csv')
duration_df.to_csv('duration_matrix.csv')


## Distancias y Tiempos aereos


In [2]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2

# Función para calcular la distancia haversiana
def haversine(coord1, coord2):
    # Coordenadas en radianes
    lon1, lat1 = radians(coord1[0]), radians(coord1[1])
    lon2, lat2 = radians(coord2[0]), radians(coord2[1])

    # Fórmula haversiana
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Radio de la Tierra en kilómetros
    R = 6371.0
    return R * c

# Cargar datos
depots_df = pd.read_csv('Depots.csv')
clients_df = pd.read_csv('Clients.csv')

# Extraer coordenadas
depots_coords = depots_df[['Longitude', 'Latitude']].values.tolist()
clients_coords = clients_df[['Longitude', 'Latitude']].values.tolist()

# Combinar coordenadas de depósitos y clientes
all_coords = depots_coords + clients_coords

# Crear la matriz de distancias haversianas
n = len(all_coords)
distance_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        distance_matrix[i, j] = haversine(all_coords[i], all_coords[j])

# Etiquetas para filas y columnas
depot_labels = [f"D{i+1}" for i in range(len(depots_coords))]
client_labels = [f"C{i+1}" for i in range(len(clients_coords))]
all_labels = depot_labels + client_labels

# Convertir la matriz en un DataFrame con etiquetas
distance_df = pd.DataFrame(distance_matrix, index=all_labels, columns=all_labels)

# Guardar la matriz en un archivo CSV
distance_df.to_csv('distance_dron_matrix.csv')

print("Matriz de distancias haversianas guardada como 'distance_matrix.csv'.")


Matriz de distancias haversianas guardada como 'distance_matrix.csv'.
